In [1]:
import covasim as cv
import numpy as np
import pandas as pd
from os.path import join
import matplotlib.pyplot as plt
import seaborn as sns

cv.options.set(dpi=100, show=False, close=True, verbose=0) # Standard options for Jupyter notebook

Covasim 3.1.1 (2021-12-06) — © 2021 by IDM


In [2]:
contact_dir = '/mnt/d/books/iitm/agentBased/codes/tn_contact_matrix/output/contact_matrix'
home_path = join(contact_dir, 'home_contact.csv')
school_path = join(contact_dir, 'school_contact.csv')
work_path = join(contact_dir, 'work_contact.csv')
community_path = join(contact_dir, 'community_contact.csv')

In [3]:
fb_dir = '/mnt/d/books/iitm/agentBased/codes/covasim/models/data/'
tile_path = join(fb_dir, 'tn_quadkey.csv')
density_path = join(fb_dir, 'pop_density.csv')
mobility_path = '/mnt/d/books/iitm/agentBased/data/fb/mobility/'

In [4]:
case_datafile = '/mnt/d/books/iitm/agentBased/data/tn/incovid19/covasim/state_datafile.csv'

In [5]:
# start_day, end_day = '2020-06-01', '2020-06-30'

start_day, end_day = '2020-06-01', '2020-06-07'

In [6]:
df = pd.read_csv(case_datafile, delimiter=',')
df = df[(df['date']>=start_day) & (df['date']<=end_day)]

refactor_file = '/mnt/d/books/iitm/agentBased/data/tn/incovid19/covasim/tmp_datafile.csv'
df.to_csv(refactor_file, sep=',', index=False, header=True)
df.head(n=5)

,date,new_tests,new_diagnoses,new_recoveries,new_deaths
86,2020-06-01,11377,1162,413,11
87,2020-06-02,11094,1091,536,13
88,2020-06-03,14101,1286,610,8
89,2020-06-04,16447,1384,585,15
90,2020-06-05,15692,1438,861,12


In [7]:
asymptotic_rate = 0.3
pop_infected = (22333 - 12757) * (1 + asymptotic_rate)

In [8]:
age_based_infection = [0.045455, 0.009091, 0.014773, 0.00000, 0.051136, 0.022727, 0.068182, 0.102273, 0.211364, 0.029924, 0.120833, 0.117424, 0.04697, 0.063636, 0.000000, 0.096212]
tile_infection = [0.        , 0.00430416, 0.00215208, 0.04925873, 0.00023912,
       0.00454328, 0.03778097, 0.02223816, 0.00621712, 0.15845688,
       0.01721664, 0.17663   , 0.14506616, 0.01602104, 0.0215208 ,
       0.0143472 , 0.04710665, 0.00701419, 0.04949785, 0.05499761,
       0.02662203, 0.01960784, 0.01259366, 0.00557947, 0.01769488,
       0.0263032 , 0.00557947, 0.00765184, 0.02941176, 0.0143472 ,
       0.        ]

In [9]:
pop_infected

12448.800000000001

In [ ]:
pars = dict(
    start_day = start_day,
    end_day = end_day,

    pop_type = 'hybrid', # Use a more realistic population model
    location = 'India-TamilNadu', # Use population characteristics for Tamil-Nadu

    pop_size = 100_000,
    pop_scale = 1,
    rescale = True,
    pop_infected = pop_infected,

    home_matrix=home_path,
    school_matrix = school_path,
    work_matrix = work_path,
    community_matrix = community_path,

    tiles = tile_path,
    pop_density = density_path,
    mobility = mobility_path,

    use_waning = True,
    dynam_layer={'c': False},

    init_infection={'tiles':tile_infection, 'ages':age_based_infection},
    
    beta=0.030216941498219296,
    rel_death_prob=2.6190536217375318,

    rand_seed=2
)

In [ ]:
# Compare matrix model

# sim = cv.Sim(pars, datafile=refactor_file, interventions=cv.test_num(daily_tests='data'))
sim = cv.Sim(pars)
sim.run(until='2020-06-04')

# sim.plot(to_plot=['cum_infections', 'cum_reinfections']) # for 1lakhs

In [ ]:
sim.results_ready

In [ ]:
# 50_000 and 2 pop_scale 

pars = dict(
    start_day = start_day,
    end_day = end_day,

    pop_type = 'matrix', # Use a more realistic population model
    location = 'India-TamilNadu', # Use population characteristics for Tamil-Nadu

    pop_size = 50_000,
    pop_scale = 2,
    rescale = True,
    pop_infected = pop_infected,

    home_matrix=home_path,
    school_matrix = school_path,
    work_matrix = work_path,
    community_matrix = community_path,

    tiles = tile_path,
    pop_density = density_path,
    mobility = mobility_path,

    use_waning = True,
    dynam_layer={'c': False},

    init_infection={'tiles':tile_infection, 'ages':age_based_infection},
    
    beta=0.030216941498219296,
    rel_death_prob=2.6190536217375318,

    rand_seed=2
)

In [ ]:
# Compare matrix model

# sim = cv.Sim(pars, datafile=refactor_file, interventions=cv.test_num(daily_tests='data'))
sim = cv.Sim(pars)
sim.run(until='2020-06-04')

sim.plot(to_plot=['cum_infections', 'cum_reinfections']) # for 50lakhs *2 

In [ ]:
date_v = sim.datevec.tolist()

In [ ]:
sim.result_keys()

In [10]:
!pip install optuna

In [23]:
from datetime import datetime, timedelta

s = datetime.strptime('2020-04-01', '%Y-%m-%d') 
e = s + timedelta(days=6)
e.strftime('%Y-%m-%d')

'2020-04-07'

In [12]:
# 50_000 and 2 pop_scale 

pars = dict(
    start_day = start_day,
    end_day = end_day,

    pop_type = 'matrix', # Use a more realistic population model
    location = 'India-TamilNadu', # Use population characteristics for Tamil-Nadu

    pop_size = 50_000,
    pop_scale = 2,
    rescale = True,
    pop_infected = pop_infected,

    home_matrix=home_path,
    school_matrix = school_path,
    work_matrix = work_path,
    community_matrix = community_path,

    tiles = tile_path,
    pop_density = density_path,
    mobility = mobility_path,

    use_waning = True,
    dynam_layer={'c': True},

    init_infection={'tiles':tile_infection, 'ages':age_based_infection},
    
    beta=0.030216941498219296,
    rel_death_prob=2.6190536217375318,

    rand_seed=2
)

In [16]:
import sciris as sc

pars['beta'] = 0.015
pars['rel_death_prob'] = 1.0


close_school = cv.clip_edges(days=0, changes=0, layers='s')
reduce_work  = cv.clip_edges(days=0, changes=0.3, layers='w')
reduce_community = cv.clip_edges(days=0, changes=0.3, layers='c')
testing      = cv.test_num(daily_tests='data')

sim = cv.Sim(pars=pars, datafile=refactor_file, interventions=[close_school, reduce_work, reduce_community, testing])

calib_pars = dict(
    beta           = [pars['beta'], 0.005, 0.20],
    rel_death_prob = [pars['rel_death_prob'], 0.5, 3.0],
    work_changes   = [0.3, 0.0, 1.0],
    community_changes = [0.3, 0.0, 1.0]
)


def set_clip_edges(sim, calib_pars):
    _, work, community = sim.get_interventions(cv.clip_edges)
    work.changes = calib_pars['work_changes']
    community.changes = calib_pars['community_changes']
    return sim


fit_args = dict(
    weights = {'new_diagnoses': 0.25, 'new_recoveries': 0.25 , 'new_deaths': 0.5},
    keys = ['new_diagnoses', 'new_recoveries', 'new_deaths'],
    normalize=False,
    use_squared=True,
    as_scalar='mean',
    die=False
)

n_trials = 2
n_workers = 1
calib = sim.calibrate(calib_pars=calib_pars, custom_fn=set_clip_edges, fit_args=fit_args, n_trials=n_trials, n_workers=n_workers)

Removed existing calibration covasim_calibration.db


[I 2022-04-14 10:33:15,822] A new study created in RDB with name: covasim_calibration
[I 2022-04-14 10:33:24,452] Trial 0 finished with value: 257350.35714285716 and parameters: {'beta': 0.07942943083555327, 'rel_death_prob': 0.9219840895740584, 'work_changes': 0.21827321529036026, 'community_changes': 0.6715167408832268}. Best is trial 0 with value: 257350.35714285716.
[I 2022-04-14 10:33:31,208] Trial 1 finished with value: 254928.5 and parameters: {'beta': 0.030686877989562985, 'rel_death_prob': 0.8565375327022496, 'work_changes': 0.8428449820389515, 'community_changes': 0.413448493118449}. Best is trial 1 with value: 254928.5.


Making results structure...
Processed 2 trials; 0 failed
Removed existing calibration covasim_calibration.db
Calibration for 2 total trials completed in 16.5 s.

Initial parameter values:
#0. beta:              0.015
#1. rel_death_prob:    1.0
#2. work_changes:      0.3
#3. community_changes: 0.3

Best parameter values:
#0. beta:              0.030686877989562985
#1. community_changes: 0.413448493118449
#2. rel_death_prob:    0.8565375327022496
#3. work_changes:      0.8428449820389515

Mismatch before calibration: 257375
Mismatch after calibration:  254928
Percent improvement:         1.0%


In [41]:
after = calib.after.copy()

In [ ]:
# # Custom objective running using scipy

# import scipy

# def objective(x, n_runs=10):
#     print(f'Running sim for beta={x[0]}, rel_death_prob={x[1]}')
#     pars = dict(
#         start_day = start_day,
#         end_day = end_day,

#         pop_type = 'matrix', # Use a more realistic population model
#         location = 'India-TamilNadu', # Use population characteristics for Tamil-Nadu

#         pop_size = 700_000,
#         pop_scale = 10,
#         rescale = True,
#         pop_infected = pop_infected,

#         home_matrix=home_path,
#         school_matrix = school_path,
#         work_matrix = work_path,
#         community_matrix = community_path,

#         tiles = tile_path,
#         pop_density = density_path,
#         mobility = mobility_path,

#         use_waning = True,
#         dynam_layer={'c': True},

#         init_infection={'tiles':tile_infection, 'ages':age_based_infection},

#         beta           = x[0],
#         rel_death_prob = x[1],
#         verbose        = 0,
#     )
#     sim = cv.Sim(pars=pars, datafile=refactor_file, interventions=cv.test_num(daily_tests='data'))
#     msim = cv.MultiSim(sim)
#     msim.run(n_runs=n_runs)
#     mismatches = []
#     for sim in msim.sims:
#         fit = sim.compute_fit()
#         mismatches.append(fit.mismatch)
#     mismatch = np.mean(mismatches)
#     return mismatch

# guess = [0.015, 1] # Initial guess of parameters -- beta and relative death probability
# pars = scipy.optimize.minimize(objective, x0=guess, method='nelder-mead') # Run the optimization

In [ ]:
# Run calibrated model

In [ ]:
start_day, end_day = '2020-06-01', '2020-07-31'

df = pd.read_csv(case_datafile, delimiter=',')
df = df[(df['date']>=start_day) & (df['date']<=end_day)]

after_calib_file = '/mnt/d/books/iitm/agentBased/data/tn/incovid19/covasim/tmp_datafile_after_calibration.csv'
df.to_csv(after_calib_file, sep=',', index=False, header=True)
df.head(n=5)

In [ ]:
pars = dict(
    start_day = start_day,
    end_day = end_day,

    pop_type = 'matrix', # Use a more realistic population model
    location = 'India-TamilNadu', # Use population characteristics for Tamil-Nadu

    pop_size = 1_000_000,
    pop_scale = 7,
    rescale = True,
    pop_infected = pop_infected,

    home_matrix=home_path,
    school_matrix = school_path,
    work_matrix = work_path,
    community_matrix = community_path,

    tiles = tile_path,
    pop_density = density_path,
    mobility = mobility_path,

    use_waning = True,
    dynam_layer={'c': True},

    init_infection={'tiles':tile_infection, 'ages':age_based_infection},
    
    beta=0.030216941498219296,
    rel_death_prob=2.6190536217375318,

    rand_seed=2
)

In [ ]:
sim = cv.Sim(pars, datafile=after_calib_file, interventions=cv.test_num(daily_tests='data'))
sim.run()

In [ ]:
sim.plot(to_plot=['new_diagnoses', 'new_deaths', 'new_recoveries'])

In [ ]:
age_analyzer = cv.age_histogram(sim=sim, states=['severe', 'dead', 'tested', 'diagnosed'])
age_analyzer.plot()